# Multi-task Tutorial
----

In this example, I will show you the method how to do mutl-task(multi-label) training, after that I will show how to predict using saved model.

This tutorial based on mxnet examples under "/example/multi-task" folder, most of the code I use based on it, but I will try to make clear some questions I met during try such example.

let's get started!

----

At the very begining, I would like to make clear what kind of "multi-task" we will do. Basically, I will use MNIST dataset, try to do 10digits classification, but this time I added a completely new label to MNIST dataset, describe whether the digit is larger than 5. so our network do two tasks now, one is digit classification, the other is true/false classification.

In order to realise this design, we need to modify the network definition and data iterator in "multi-task" example.


Following code do some preparation, just same as "muti-task" example.

In [19]:
import sys
sys.path.insert(0, "../../python/")
import mxnet as mx
import numpy as np
import time
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Then I change the network definition for real multi-task case. Please take care of fc4,sm2 espacially, they are different from original example. fc4 num_hidden=2 is on the basis of requirement of our second task, a true/false classification. If you have a different task, please at least modify num_hidden to adapt your own's request.

In [20]:
def build_network():
    data = mx.symbol.Variable('data')
    fc1 = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=128)
    act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
    fc2 = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 64)
    act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
    fc3 = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=10)
    sm1 = mx.symbol.SoftmaxOutput(data = fc3, name = 'softmax1')
    fc4 = mx.symbol.FullyConnected(data = act2, name='fc4', num_hidden=2)
    sm2 = mx.symbol.SoftmaxOutput(data = fc4, name = 'softmax2')

    softmax = mx.symbol.Group([sm1, sm2])

    return softmax

Next step, I modify the data iterator to support output two labels, this time I just try to do minimum modification on minist_iterator, in the end of this tutorial, I will show you how to generate and load data from .rec files through  ImageRecordIter.

In [21]:
class Multi_mnist_iterator(mx.io.DataIter):
    '''multi label mnist iterator'''

    def __init__(self, data_iter):
        super(Multi_mnist_iterator, self).__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size

    @property
    def provide_data(self):
        return self.data_iter.provide_data

    @property
    def provide_label(self):
        provide_label = self.data_iter.provide_label[0]
        # Different labels should be used here for actual application
        return [('softmax1_label', provide_label[1]), \
                ('softmax2_label', provide_label[1])]

    def hard_reset(self):
        self.data_iter.hard_reset()

    def reset(self):
        self.data_iter.reset()

    def next(self):
        batch = self.data_iter.next()
        label = batch.label[0] # the original label here
        label2 = mx.nd.array(label.asnumpy()>5).astype('float32') # new label based on the original label, output 0 or 1 here
        return mx.io.DataBatch(data=batch.data, label=[label, label2], \
                pad=batch.pad, index=batch.index)

Accuracy calculate code is exactly same as "multi-task" sample.

In [15]:
class Multi_Accuracy(mx.metric.EvalMetric):
    """Calculate accuracies of multi label"""

    def __init__(self, num=None):
        super(Multi_Accuracy, self).__init__('multi-accuracy', num)

    def update(self, labels, preds):
        mx.metric.check_label_shapes(labels, preds)

        if self.num != None:
            assert len(labels) == self.num

        for i in range(len(labels)):
            pred_label = mx.nd.argmax_channel(preds[i]).asnumpy().astype('int32')
            label = labels[i].asnumpy().astype('int32')

            mx.metric.check_label_shapes(label, pred_label)

            if i == None:
                self.sum_metric += (pred_label.flat == label.flat).sum()
                self.num_inst += len(pred_label.flat)
            else:
                self.sum_metric[i] += (pred_label.flat == label.flat).sum()
                self.num_inst[i] += len(pred_label.flat)

Data iterator is also exactly same as the sample.

In [16]:
# pylint: skip-file
""" data iterator for mnist """
import sys
import os

# code to automatically download dataset
sys.path.append( "../../tests/python/common")
import get_data
import mxnet as mx

def mnist_iterator(batch_size, input_shape):
    """return train and val iterators for mnist"""
    # download data
    get_data.GetMNIST_ubyte()
    flat = False if len(input_shape) == 3 else True

    train_dataiter = mx.io.MNISTIter(
        image="data/train-images-idx3-ubyte",
        label="data/train-labels-idx1-ubyte",
        input_shape=input_shape,
        batch_size=batch_size,
        shuffle=True,
        flat=flat)

    val_dataiter = mx.io.MNISTIter(
        image="data/t10k-images-idx3-ubyte",
        label="data/t10k-labels-idx1-ubyte",
        input_shape=input_shape,
        batch_size=batch_size,
        flat=flat)

    return (train_dataiter, val_dataiter)

Next, do some settings and load data, here I add some code to save trained model after each epoch. 
Also, please pay attation to the status of ipython notebook, make sure it finished and then continue. MNIST data will be downloaded if it is not exists. It can take several minutes due to bad network condition.

In [17]:
batch_size=100
num_epochs=100
device = mx.gpu(0) # use mx.cpu(0) if you don't have a gpu card properly installed.
lr = 0.01
save_model_prefix = "mnist_multitask" # model save prefix

network = build_network()

train, val = mnist_iterator(batch_size=batch_size, input_shape = (784,))
train = Multi_mnist_iterator(train)
val = Multi_mnist_iterator(val)

checkpoint = mx.callback.do_checkpoint(save_model_prefix) # checkpoint callback save trained parameters.

Next set the checkpoint to model.fit function and we can start training.

In [23]:
model = mx.model.FeedForward(
    ctx                = device,
    symbol             = network,
    num_epoch          = num_epochs,
    learning_rate      = lr,
    momentum           = 0.9,
    wd                 = 0.00001,
    initializer        = mx.init.Xavier(factor_type="in", magnitude=2.34))

model.fit(
    X                  = train,
    eval_data          = val,
    eval_metric        = Multi_Accuracy(num=2),
    batch_end_callback = mx.callback.Speedometer(batch_size, 50),
    epoch_end_callback = checkpoint # here we set call checkpoint method after each epoch ended.
    ) 

INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [50]	Speed: 80117.97 samples/sec	Train-multi-accuracy_0=0.452000
INFO:root:Epoch[0] Batch [50]	Speed: 80117.97 samples/sec	Train-multi-accuracy_1=0.685600
INFO:root:Epoch[0] Batch [100]	Speed: 79673.58 samples/sec	Train-multi-accuracy_0=0.793600
INFO:root:Epoch[0] Batch [100]	Speed: 79673.58 samples/sec	Train-multi-accuracy_1=0.835200
INFO:root:Epoch[0] Batch [150]	Speed: 96240.69 samples/sec	Train-multi-accuracy_0=0.857600
INFO:root:Epoch[0] Batch [150]	Speed: 96240.69 samples/sec	Train-multi-accuracy_1=0.873400
INFO:root:Epoch[0] Batch [200]	Speed: 98771.31 samples/sec	Train-multi-accuracy_0=0.883400
INFO:root:Epoch[0] Batch [200]	Speed: 98771.31 samples/sec	Train-multi-accuracy_1=0.917400
INFO:root:Epoch[0] Batch [250]	Speed: 99322.36 samples/sec	Train-multi-accuracy_0=0.897600
INFO:root:Epoch[0] Batch [250]	Speed: 99322.36 samples/sec	Train-multi-accuracy_1=0.928000
INFO:root:Epoch[0] Batch [300]	Speed: 86871.69 sample

As you can see, after 100 epoch, the validation accuracy of task1 is around 0.979300, the validation accuracy of task2 is around 0.985100. (the result on your environment may different with this, but should be very close.)

Since we got the trained prameters, let us have a test. I will show you how to predict a test image using trained parameters.
Let us display the test image firstly.

In [24]:
from IPython.display import Image
Image(filename='2.jpg')

The predict method as below

In [25]:
sys.path.insert(0, "../../python/")
sys.path.append("../../amalgamation/python/")
import mxnet as mx
import numpy as np
from mxnet_predict import Predictor
from skimage import io
from skimage.color import rgb2gray

def predict(params, symbol, image):
    predictor = Predictor(open(symbol).read(), open(params).read(), {'data':(1 , 1, 28, 28)},'gpu',0) #use cpu if you don't have a gpu card
    img = io.imread(image)
    img = rgb2gray(img)
    
    predictor.forward(data=img)
    # for first task
    prob = predictor.get_output(0)
    pred = np.argsort(prob[0])[::-1]
    # label_id (same as label if MNIST), score
    print pred[0], prob[0][pred[0]]
    # for second task
    prob2 = predictor.get_output(1)
    pred2 = np.argsort(prob2[0])[::-1]
    # label_id (same as label if MNIST), score
    print pred2[0],  prob2[0][pred2[0]]

In [26]:
symbol_file = "./mnist_multitask-symbol.json"
params_file = "./mnist_multitask-0100.params"
image_file = "2.jpg"
predict(params_file, symbol_file, image_file)

2 0.535227
0 0.999994


As we can see, for the task one, predict output 2 0.535227 which means the predict result is 2 and confidence is 0.535227
for the task two, predict output 0 0.999994 which means the predict result is "not >5" and confidence is 0.999994

Somebody may have question about data iterator, since in my previous tutorials, I did't load data from scratch.
Now I will show you the method to load data from scratch.

My dataset located under mxnet/example/notebooks folder and looks like this

In [12]:
out =!tree -i -f `pwd`/dataset|grep .jpg
out

['/home/haria/mxnet/example/notebooks/dataset/female/PersonA/00e93901213fb80e4c0c11df34d12f2eb83894dc_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/00e93901213fb80e747739df34d12f2eb9389417_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0b46f21fbe096b63a2019b510e338744ebf8ac93_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0b46f21fbe096b63c7c9785a0e338744eaf8ace3_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0b55b319ebc4b745938fa91ccdfc1e178a8215a4_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0bd162d9f2d3572c16a7b7d28813632762d0c372_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0bd162d9f2d3572c1d66b0d58813632762d0c336_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0bd162d9f2d3572c26c547de8813632762d0c389_ok.jpg',
 '/home/haria/mxnet/example/notebooks/dataset/female/PersonA/0dd7912397dda1445f1b21bab0b7d0a20cf486ac_ok.jpg',
 

I generate train.txt and val.txt base on previous command output. following two command show the content of train.txt and val.txt

In [13]:
out = !cat train.txt
out

['1 0 1 /female/PersonA/00e93901213fb80e4c0c11df34d12f2eb83894dc_ok.jpg',
 '2 0 1 /female/PersonA/00e93901213fb80e747739df34d12f2eb9389417_ok.jpg',
 '3 0 1 /female/PersonA/0b46f21fbe096b63a2019b510e338744ebf8ac93_ok.jpg',
 '4 0 1 /female/PersonA/0b46f21fbe096b63c7c9785a0e338744eaf8ace3_ok.jpg',
 '5 0 1 /female/PersonA/0b55b319ebc4b745938fa91ccdfc1e178a8215a4_ok.jpg',
 '6 0 1 /female/PersonA/0bd162d9f2d3572c16a7b7d28813632762d0c372_ok.jpg',
 '7 0 1 /female/PersonA/0bd162d9f2d3572c1d66b0d58813632762d0c336_ok.jpg',
 '8 0 1 /female/PersonA/0bd162d9f2d3572c26c547de8813632762d0c389_ok.jpg',
 '9 0 1 /female/PersonA/0dd7912397dda1445f1b21bab0b7d0a20cf486ac_ok.jpg',
 '10 0 1 /female/PersonA/0df3d7ca7bcb0a469bbe66276963f6246b60af14_ok.jpg',
 '11 0 1 /female/PersonA/0e2442a7d933c8953bbb5376d31373f0830200f2_ok.jpg',
 '12 0 1 /female/PersonA/0e2442a7d933c8955b62f365d31373f08202008b_ok.jpg',
 '13 0 1 /female/PersonA/0e2442a7d933c8956a918066d31373f08202003b_ok.jpg',
 '14 0 1 /female/PersonA/0e2442a7d

In [14]:
out =!cat val.txt
out

['41 0 1 /female/PersonA/4a36acaf2edda3ccf691612103e93901213f92be_ok.jpg',
 '42 0 1 /female/PersonA/4b90f603738da9777c510c04b351f8198618e3a7_ok.jpg',
 '43 0 1 /female/PersonA/4b90f603738da977e3349821b251f8198618e3a5_ok.jpg',
 '44 0 1 /female/PersonA/4bed2e738bd4b31c70b6b8b285d6277f9e2ff835_ok.jpg',
 '45 0 1 /female/PersonA/4e4a20a4462309f78cf1861f700e0cf3d7cad624_ok.jpg',
 '46 0 1 /female/PersonA/4e4a20a4462309f7a7059b1c700e0cf3d7cad651_ok.jpg',
 '47 0 1 /female/PersonA/5ab5c9ea15ce36d34dcc081f39f33a87e850b1d1_ok.jpg',
 '48 0 1 /female/PersonA/5bafa40f4bfbfbed0e284f777af0f736afc31f40_ok.jpg',
 '49 0 1 /female/PersonA/5d6034a85edf8db1477c596e0b23dd54564e746d_ok.jpg',
 '50 0 1 /female/PersonA/5d6034a85edf8db1b569576a0b23dd54564e747c_ok.jpg',
 '91 1 1 /female/PersonB/2cf5e0fe9925bc311ec762905cdf8db1cb13701b_ok.jpg',
 '92 1 1 /female/PersonB/2e2eb9389b504fc211093a08e7dde71190ef6d6d_ok.jpg',
 '93 1 1 /female/PersonB/2e2eb9389b504fc2111f3a08e7dde71190ef6d7f_ok.jpg',
 '94 1 1 /female/PersonB/

Next we convert image list file to .rec file using im2rec tools. 

the command is: ~/mxnet/bin/im2rec  train.txt ./dataset train.rec label_width=2 resize=224

the options described as below:

~/mxnet/bin/im2rec    binary tool, if you can't find such tool, please check the compilation of mxnet.

 train.txt    image list file, format as I list above
 
 ./dataset    image base folder, ex, for ./dataset/female/PersonA/00e93901213fb80e4c0c11df34d12f2eb83894dc_ok.jpg the base folder is ./dataset
 
 train.rec    the output .rec file
 
 label_width=2     the label_width, since we have two labels, use 2 override the deafult 1.
 
 resize=224    the output shape of im2rec

Now we convert train.txt to train.rec

In [15]:
out = !~/mxnet/bin/im2rec  train.txt ./dataset train.rec label_width=2 resize=224
out

['[13:38:17] tools/im2rec.cc:94: New Image Size: Short Edge 224',
 '[13:38:17] tools/im2rec.cc:107: Encoding is .jpg',
 '[13:38:17] tools/im2rec.cc:153: Write to output: train.rec',
 '[13:38:17] tools/im2rec.cc:155: Output: train.rec',
 '[13:38:17] tools/im2rec.cc:168: JPEG encoding quality: 80',
 '[13:38:17] tools/im2rec.cc:254: Total: 320 images processed, 0.288873 sec elapsed']

Then we convert val.txt to val.rec

In [16]:
out = !~/mxnet/bin/im2rec  val.txt ./dataset val.rec label_width=2 resize=224
out

['[13:38:18] tools/im2rec.cc:94: New Image Size: Short Edge 224',
 '[13:38:18] tools/im2rec.cc:107: Encoding is .jpg',
 '[13:38:18] tools/im2rec.cc:153: Write to output: val.rec',
 '[13:38:18] tools/im2rec.cc:155: Output: val.rec',
 '[13:38:18] tools/im2rec.cc:168: JPEG encoding quality: 80',
 '[13:38:18] tools/im2rec.cc:254: Total: 80 images processed, 0.0746553 sec elapsed']

Now it is ready to train using generated train.rec and val.rec. I update data iterator as below.

In [8]:
def mnist_iterator(batch_size, input_shape):
    data_shape = input_shape
    train = mx.io.ImageRecordIter(
        path_imglist= "./train.txt",  # you have to specify path_imglist when label_width larger than 2.
        path_imgrec = "./train.rec",
        mean_img    = "./mean.bin", # it is OK no such file exists before train, the mxnet will generated it if not exists
        data_shape  = data_shape,
        batch_size  = batch_size,
        rand_crop   = True,
        rand_mirror = True,
        label_width = 2          # specify label_width = 2 here
        )

    val = mx.io.ImageRecordIter(
        path_imglist= "./val.txt", 
        path_imgrec = "./val.rec",
        mean_img    = "./mean.bin",
        rand_crop   = False,
        rand_mirror = False,
        data_shape  = data_shape,
        batch_size  = batch_size,
        label_width = 2
        )

    return (train, val)

class Multi_mnist_iterator(mx.io.DataIter):
    '''multi label mnist iterator'''

    def __init__(self, data_iter):
        super(Multi_mnist_iterator, self).__init__()
        self.data_iter = data_iter
        self.batch_size = self.data_iter.batch_size

    @property
    def provide_data(self):
        return self.data_iter.provide_data

    @property
    def provide_label(self):
        provide_label = self.data_iter.provide_label[0]
        # Different labels should be used here for actual application
        return [('softmax1_label', (self.batch_size,)), \
                ('softmax2_label', (self.batch_size,))]

    def hard_reset(self):
        self.data_iter.hard_reset()

    def reset(self):
        self.data_iter.reset()

    def next(self):
        batch = self.data_iter.next()
        label = batch.label[0]
        label1,label2 = label.T.asnumpy() 
        label1=mx.nd.array(label1)
        label2=mx.nd.array(label2)
        return mx.io.DataBatch(data=batch.data, label=[label1, label2], \
                pad=batch.pad, index=batch.index)

In [9]:
def build_network(): # I use vgg network architecture, I just prepared 400 samples, if use default network, not converge at all.
    data = mx.symbol.Variable(name="data")
    # group 1
    conv1_1 = mx.symbol.Convolution(data=data, kernel=(3, 3), pad=(1, 1), num_filter=64, name="conv1_1")
    relu1_1 = mx.symbol.Activation(data=conv1_1, act_type="relu", name="relu1_1")
    pool1 = mx.symbol.Pooling(data=relu1_1, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool1")
    # group 2
    conv2_1 = mx.symbol.Convolution( data=pool1, kernel=(3, 3), pad=(1, 1), num_filter=128, name="conv2_1")
    relu2_1 = mx.symbol.Activation(data=conv2_1, act_type="relu", name="relu2_1")
    pool2 = mx.symbol.Pooling( data=relu2_1, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool2")
    # group 3
    conv3_1 = mx.symbol.Convolution( data=pool2, kernel=(3, 3), pad=(1, 1), num_filter=256, name="conv3_1")
    relu3_1 = mx.symbol.Activation(data=conv3_1, act_type="relu", name="relu3_1")
    conv3_2 = mx.symbol.Convolution( data=relu3_1, kernel=(3, 3), pad=(1, 1), num_filter=256, name="conv3_2")
    relu3_2 = mx.symbol.Activation(data=conv3_2, act_type="relu", name="relu3_2")
    pool3 = mx.symbol.Pooling( data=relu3_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool3")
    # group 4
    conv4_1 = mx.symbol.Convolution( data=pool3, kernel=(3, 3), pad=(1, 1), num_filter=512, name="conv4_1")
    relu4_1 = mx.symbol.Activation(data=conv4_1, act_type="relu", name="relu4_1")
    conv4_2 = mx.symbol.Convolution( data=relu4_1, kernel=(3, 3), pad=(1, 1), num_filter=512, name="conv4_2")
    relu4_2 = mx.symbol.Activation(data=conv4_2, act_type="relu", name="relu4_2")
    pool4 = mx.symbol.Pooling( data=relu4_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool4")
    # group 5
    conv5_1 = mx.symbol.Convolution( data=pool4, kernel=(3, 3), pad=(1, 1), num_filter=512, name="conv5_1")
    relu5_1 = mx.symbol.Activation(data=conv5_1, act_type="relu", name="relu5_1")
    conv5_2 = mx.symbol.Convolution( data=relu5_1, kernel=(3, 3), pad=(1, 1), num_filter=512, name="conv5_2")
    relu5_2 = mx.symbol.Activation(data=conv5_2, act_type="relu", name="conv1_2")
    pool5 = mx.symbol.Pooling(  data=relu5_2, pool_type="max", kernel=(2, 2), stride=(2,2), name="pool5")
    # group 6
    flatten = mx.symbol.Flatten(data=pool5, name="flatten")
    fc6 = mx.symbol.FullyConnected(data=flatten, num_hidden=4096, name="fc6")
    relu6 = mx.symbol.Activation(data=fc6, act_type="relu", name="relu6")
    drop6 = mx.symbol.Dropout(data=relu6, p=0.5, name="drop6")
    # group 7
    fc7 = mx.symbol.FullyConnected(data=drop6, num_hidden=4096, name="fc7")
    relu7 = mx.symbol.Activation(data=fc7, act_type="relu", name="relu7")
    drop7 = mx.symbol.Dropout(data=relu7, p=0.5, name="drop7")
    # output
    fc8a = mx.symbol.FullyConnected(data=drop7, num_hidden=8, name="fc8a")
    sm1 = mx.symbol.SoftmaxOutput(data = fc8a, name = 'softmax1')
    fc8b = mx.symbol.FullyConnected(data = drop7, name='fc8b', num_hidden=2)
    sm2 = mx.symbol.SoftmaxOutput(data = fc8b, name = 'softmax2')

    softmax = mx.symbol.Group([sm1, sm2])

    return softmax

Next it's ready to training.

In [10]:
batch_size=20
num_epochs=100
device = mx.gpu(0)
lr = 0.0001

network = build_network()
train, val = mnist_iterator(batch_size=batch_size, input_shape = (3,224,224))
train = Multi_mnist_iterator(train)
val = Multi_mnist_iterator(val)

save_model_prefix = "mnist_multitask"
checkpoint = mx.callback.do_checkpoint(save_model_prefix)

model = mx.model.FeedForward(
    ctx                = device,
    symbol             = network,
    num_epoch          = num_epochs,
    learning_rate      = lr,
    momentum           = 0.9,
    wd                 = 0.00001,
    initializer        = mx.init.Xavier(factor_type="in", magnitude=2.34))

model.fit(
    X                  = train,
    eval_data          = val,
    eval_metric        = Multi_Accuracy(num=2),
    batch_end_callback = mx.callback.Speedometer(batch_size, 50),
    epoch_end_callback = checkpoint
    )


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=4.290
INFO:root:Saved checkpoint to "mnist_multitask-0001.params"
INFO:root:Epoch[0] Validation-multi-accuracy_0=0.125000
INFO:root:Epoch[0] Validation-multi-accuracy_1=0.500000
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=3.741
INFO:root:Saved checkpoint to "mnist_multitask-0002.params"
INFO:root:Epoch[1] Validation-multi-accuracy_0=0.100000
INFO:root:Epoch[1] Validation-multi-accuracy_1=0.487500
INFO:root:Epoch[2] Resetting Data Iterator
INFO:root:Epoch[2] Time cost=3.751
INFO:root:Saved checkpoint to "mnist_multitask-0003.params"
INFO:root:Epoch[2] Validation-multi-accuracy_0=0.137500
INFO:root:Epoch[2] Validation-multi-accuracy_1=0.575000
INFO:root:Epoch[3] Resetting Data Iterator
INFO:root:Epoch[3] Time cost=3.752
INFO:root:Saved checkpoint to "mnist_multitask-0004.params"
INFO:root:Epoch[3] Validation-multi-accuracy_0=0.137500
INFO:root:Epoch[3

Since I just prepare 400 samples, the task one is not converge in a well manner, but as you can see, it works! if you have more data samples, you should get a much better output.

That is all!